# MODELS BABYYYYY - Sentiment Analysis - Big Richard Club

In [1]:
import pandas as pd
import numpy as np
import re 
import nltk 
from nltk.corpus import stopwords
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
import collections

from io import StringIO
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import chi2
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn import metrics

from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.metrics import classification_report,confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.patches as mpatches

plt.style.use('ggplot')
%matplotlib inline
pd.options.mode.chained_assignment = None  # default='warn'

## I. Import Data

In [2]:
x_train_bow = np.loadtxt("lesser_x_train_bow.txt.gz")
y_train = np.loadtxt("y_train.txt.gz")

print("Train Data Loaded")

Train Data Loaded


In [14]:
print(f"X_train shape : {x_train_bow.shape} \n Y_train shape : {y_train.shape}")

X_train shape : (41157, 1000) 
Y_train shape : (41157,)


# II. Model Charlotte

In [15]:
callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    patience=5,
    restore_best_weights=True
)

In [26]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(
        input_dim=1000, # vocab of 1000 words
        output_dim=3 # 3 classes
    ), 
    tf.keras.layers.Dropout(0.5),
    
    tf.keras.layers.Conv1D(128, 5, activation = 'relu'),
    tf.keras.layers.MaxPooling1D(pool_size = 1),
    tf.keras.layers.Dropout(0.3),
    
    #tf.keras.layers.Conv1D(256, 5, activation = 'relu'),
    #tf.keras.layers.Dropout(0.3),
    
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(64, activation = 'relu'),
    tf.keras.layers.Dense(3, activation = 'softmax')
])
model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, None, 3)           3000      
                                                                 
 dropout_12 (Dropout)        (None, None, 3)           0         
                                                                 
 conv1d_6 (Conv1D)           (None, None, 128)         2048      
                                                                 
 max_pooling1d_3 (MaxPooling  (None, None, 128)        0         
 1D)                                                             
                                                                 
 dropout_13 (Dropout)        (None, None, 128)         0         
                                                                 
 bidirectional_3 (Bidirectio  (None, 64)               31104     
 nal)                                                 

In [27]:
history = model.fit(x_train_bow, y_train)

 407/1287 [========>.....................] - ETA: 8:19 - loss: 1.0456 - accuracy: 0.4400

KeyboardInterrupt: 

# 44% ACCURACY C'EST CLAQUEEEEE